In [1]:
# Get the template
from app.src.helpers.template_getter import get_template
contract = get_template('meat_sale')

print(contract.to_sym())


DOMAIN MODEL:
== Roles ==
seller: seller
- address (str): 123 Main street

buyer: buyer
- warehouse (str): 999 Central Ave


== Assets ==
perishableGood: perishableGood (None)
- quantity (number): 
- quality (MeatQuality): 

meat: meat (perishableGood)


== Events ==
delivered: delivered
- item (meat): meat

paid: paid
- from (Role): buyer
- to (Role): seller

paidLate: paidLate
- from (Role): buyer
- to (Role): seller

disclosed: disclosed



CONTRACT SPEC:

== Obligations ==
delivery: O(seller, buyer, true, Happens(delivered))

payment: O(buyer, seller, true, Happens(paid))

latePayment: Happens(Violated(obligations.payment)) -> O(buyer, seller, true, Happens(paidLate))

disclosure1: O(seller, buyer, true, not Happens(disclosed))

disclosure2: O(buyer, seller, true, not Happens(disclosed))


== Powers ==
suspendDelivery: P(seller, buyer, true, Suspended(obligations.delivery))

resumeDelivery: Happens(NEVER) -> P(buyer, seller, true, Resumed(obligations.delivery))

terminateContract:

In [2]:
obs = contract.template_strings['obligations']
pows = contract.template_strings['powers']

print('== OBLIGATIONS ==')
for x in obs:
    print(f'- {x}: {obs[x]}\n')

print('== POWERS ==')
for x in pows:
    print(f'- {x}: {pows[x]}\n')

== OBLIGATIONS ==
- delivery: The Seller shall deliver the Order in one delivery to the Buyer [DELIVERY_REFINEMENT]

- delivery_old: The Seller shall deliver the Order in one delivery [DELIVERY_TIMEFRAME] to the Buyer [DELIVERY_LOCATION]

- payment: The Buyer shall pay [PAYMENT_DETAILS] to the Seller [PAYMENT_TIMEFRAME].

- latePayment: In the event of late payment of the amount owed due, the Buyer shall pay interests equal to [INTEREST_DETAILS]

- disclosure: Both Seller and Buyer must keep the contents of this contract confidential [CONFIDENTIALITY_TIMEFRAME]

== POWERS ==
- suspendResumeDelivery: [DELIVERY_SUSPENSION_CONDITION] the Seller may suspend performance of all of its obligations under the agreement [DELIVERY_RESUMPTION_CONDITION]

- terminateContract: Any delay in delivery of the goods will not entitle the Buyer to terminate the Contract [TERMINATION_CONDITION]



In [3]:
# Select the parameter
from app.templates.meat_sale.nl_template import parameters

pki_dict = {i+1: pk for i,pk in enumerate(parameters)}

for pki in pki_dict:
    print(f'{pki} - {pki_dict[pki]}')

selected_pki = input('Select a parameter: ')
selected_pk = pki_dict[int(selected_pki)]
selected_parm = parameters[selected_pk][0]

print(f'\nParameter: {selected_pk}')

# The selected parameter will trigger some other things, e.g. which grammar/frameset to use...


1 - DELIVERY_REFINEMENT
2 - DELIVERY_TIMEFRAME
3 - DELIVERY_LOCATION
4 - PAYMENT_DETAILS
5 - PAYMENT_TIMEFRAME
6 - INTEREST_DETAILS
7 - CONFIDENTIALITY_TIMEFRAME
8 - DELIVERY_SUSPENSION_CONDITION
9 - DELIVERY_RESUMPTION_CONDITION
10 - TERMINATION_CONDITION

Parameter: DELIVERY_REFINEMENT


In [4]:
from app.src.operations.helpers.default_event_getter import DefaultEventGetter

# Extract the default event
deg = DefaultEventGetter()

# Handle the case where there is none...
default_event = deg.get(contract.contract_spec, selected_parm.config)

print(f'Default Event: {default_event.to_sym()}')


Default Event: delivered


In [ ]:
#default_event = None

In [5]:
from app.src.operations.configs import OpCode
from app.src.grammar.grammar_generator import GrammarGenerator
from app.src.grammar.grammar_config import GrammarGeneratorConfig

# Generate the grammar from the contract template
gg = GrammarGenerator()

# These should probably come from the parameter...
op_codes = [OpCode.ADD_TRIGGER, OpCode.REFINE_PREDICATE] 

grammar_config = GrammarGeneratorConfig(op_codes)

grammar_root = gg.generate(contract, grammar_config)


In [6]:
from app.src.grammar.grammar_selector import GrammarSelector
from app.src.grammar.manual_node_selector import ManualGrammarNodeSelector

# Run the selection so the user can choose the text
inner_selector = ManualGrammarNodeSelector()
gs = GrammarSelector(inner_selector)
selection = gs.select(grammar_root)


Choose an option:
- 1 if
- 2 before
- 3 after
- 4 within

Choose an option:
- 1 Enter a date
- 2 Describe an event


In [7]:
from app.src.frames.frame_checker_constuctor import FrameCheckerConstructor

frame_checker = FrameCheckerConstructor.construct()
frames = frame_checker.check_all_frames(selection.nodes)

for x in frames:
    print(type(x), x.to_text())

main_frame = frames[0]
op_code = main_frame.op_code

<class 'app.classes.frames.after_date_frame.AfterDateFrame'> after 2022/02/23


In [8]:
# Generate the Symboleo
symboleo_obj = selection.to_obj(default_event)
print(symboleo_obj.to_sym())

HappensAfter(delivered, 2022/02/23)


In [9]:
# Updating
from app.src.operations.contract_updater_builder import ContractUpdaterBuilder

contract_updater = ContractUpdaterBuilder.build()

updated_contract = contract_updater.update(op_code, contract, symboleo_obj, selected_parm.config)


In [10]:
print(updated_contract.to_sym())


DOMAIN MODEL:
== Roles ==
seller: seller
- address (str): 123 Main street

buyer: buyer
- warehouse (str): 999 Central Ave


== Assets ==
perishableGood: perishableGood (None)
- quantity (number): 
- quality (MeatQuality): 

meat: meat (perishableGood)


== Events ==
delivered: delivered
- item (meat): meat

paid: paid
- from (Role): buyer
- to (Role): seller

paidLate: paidLate
- from (Role): buyer
- to (Role): seller

disclosed: disclosed



CONTRACT SPEC:

== Obligations ==
delivery: O(seller, buyer, true, HappensAfter(delivered, 2022/02/23))

payment: O(buyer, seller, true, Happens(paid))

latePayment: Happens(Violated(obligations.payment)) -> O(buyer, seller, true, Happens(paidLate))

disclosure1: O(seller, buyer, true, not Happens(disclosed))

disclosure2: O(buyer, seller, true, not Happens(disclosed))


== Powers ==
suspendDelivery: P(seller, buyer, true, Suspended(obligations.delivery))

resumeDelivery: Happens(NEVER) -> P(buyer, seller, true, Resumed(obligations.delivery))

t